### **Part 4**
**Tasks**
* explore proper visual stimuli
  * spontaneous state
  * (done) evoked state: drifting
* Communication subspaces across different areas
  * (done) correlation
  * (done) dimensionality
    * intra_areas
    * inter_areas
* Information flow across subspaces
  * (done) direction: population correlation as the function of the time delay between areas ($t_2$-$t_1$)
  * intensity (correlation intensity)
* Hierarchy of information flow  across these subspaces
  * direction
    * (done) inter_areas cca_delay
    * *intra_areas cca_delay (layer_dependent)*
    * (done) feedfward and feedback
    * (done) input/output dimensions
  * function validation
* Relation between different information flow channels
  * *subspace & layer_dependent*
  * shared dimensionility
  * (done) generalize canonical dimension across subspaces
  * (done) subspaces angle
  * CCA across subspaces
  * predictive coding
* (to do) State dependent geometrical properties of these channels
* (to do) Dynamical system across areas modeled by multi-region RNN network

*****************
(doing) 
* clear up code, make them beauty
* speed up code, make them faster
* generize code across areas and sessions/mice
****************

In [1]:
import os
import torch
import cupy as cp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import subspace_angles
from sklearn.model_selection import KFold
from sklearn.decomposition import FactorAnalysis
from sklearn.cross_decomposition import CCA
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

In [23]:
def set_device(i):
  """
  Set the device. CUDA if available, CPU otherwise
  """
  device = "cuda" if torch.cuda.is_available() else "cpu"
  if device != "cuda":
    print("GPU is not enabled in this notebook.")
  else:
    print("GPU is enabled in this notebook.")
    if torch.cuda.device_count() >= i + 1:
      device = f'cuda:{i}'
  return device

DEVICE = set_device(0)
print('Num of gpu:', torch.cuda.device_count())

x = torch.tensor([0, 1, 2], device=DEVICE)
y = torch.tensor([3, 4, 5], device="cpu")
print('example:', y)
y = y.to(DEVICE)  # alternatively, you can use y = y.cuda()
print('example:', y)
x = x.to('cuda:1')
print('example:', x)

GPU is enabled in this notebook.
Num of gpu: 4
example: tensor([3, 4, 5])
example: tensor([3, 4, 5], device='cuda:0')
example: tensor([0, 1, 2], device='cuda:1')


In [2]:
# basepath = "E:\Allensdk_data\local\ecephys_cache_dir"
basepath = "/home/jialab/Allensdk_data/local/ecephys_cache_dir/"
manifest_path = os.path.join(basepath, "manifest.json")
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

sessions = cache.get_session_table()
session_id = 799864342
session = cache.get_session_data(session_id)

drift_stim_table = session.get_stimulus_table('drifting_gratings')
drift_stim_table.head()

### Task7 Layer dependent information flow

### Task8 Relation between area_subspace & layer_channel